# Load Data

In [1]:
import scipy
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
from PIL import ImageFont, ImageDraw, Image
import tensorflow as tf
from tensorflow.python.framework.ops import EagerTensor
import random
from tensorflow.keras.layers import Dense, Input, Flatten, Flatten, GlobalMaxPooling2D
import tensorflow.keras.applications.vgg16 as vgg16
from tensorflow.keras.models import load_model, Model
from yad2k.models.keras_yolo import yolo_head
from yad2k.utils.utils import draw_boxes, get_colors_for_classes, scale_boxes, read_classes, read_anchors, preprocess_image

2022-09-08 15:59:23.086141: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-09-08 15:59:23.594035: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-08 15:59:23.594045: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
data = scipy.io.loadmat('imdb.mat')
data = data["imdb"][0][0]

In [3]:
dob = data[0][0]
photoTaken = data[1][0]
fullPath = data[2][0]
gender = data[3][0]
name = data[4][0]
faceLocation = data[5][0]  
faceScore = data[6][0]
secondFaceScore = data[7][0]

In [4]:
# data_out = []
X = []
Y = []
IMAGE_SIZE = 256
print(len(dob))
for i in range(len(dob)):
    # test in dataset 01
    if(fullPath[i][0][0: 2] != "00"): 
        continue
        
    image = {
        "dob": dob[i],
        "fullPath": fullPath[i],
        "faceLocation": faceLocation[i],
        "faceScore": faceScore[i]
    }
    try:
        original_image, imageEncoded = preprocess_image("train_data/00/" + image["fullPath"][0], model_image_size = (IMAGE_SIZE, IMAGE_SIZE))    
        width, height = original_image.size
        left, top, right, bottom = image["faceLocation"][0][0] / width, image["faceLocation"][0][1] / height, image["faceLocation"][0][2] / width, image["faceLocation"][0][3] / height
        if(np.array(imageEncoded[0]).shape == ((IMAGE_SIZE, IMAGE_SIZE))):
            original_image.save("error_image.jpg")
            continue
        X.append(imageEncoded[0])
        Y.append((scipy.special.expit(image["faceScore"]), left, top , right, bottom)) 
    except:
        continue
# X = np.array(X, dtype=object)
# Y = np.array(Y, dtype=object)
print("NUM OF TRAINING EXAMPLES", len(X))
train_dataset = tf.data.Dataset.from_tensor_slices((X, Y))

BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

# test_dataset = tf.data.Dataset.from_tensor_slices((test_examples, test_labels))

# X.shape

460723
NUM OF TRAINING EXAMPLES 3657


2022-09-08 16:01:23.033932: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-08 16:01:23.033945: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-08 16:01:23.033957: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (huwngnosleep): /proc/driver/nvidia/version does not exist
2022-09-08 16:01:23.044001: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Model

In [5]:
inputLayer = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
vgg_model = vgg16.VGG16(include_top=False)
for layer in vgg_model.layers:
    layer.trainable = False

vgg = vgg_model(inputLayer)

# f1 = GlobalMaxPooling2D()(vgg)
# f1 = Dense(2048, activation='relu')(f1)
# f1 = Dense(1, activation='sigmoid')(f1)

f2 = GlobalMaxPooling2D()(vgg)
f2 = Dense(2048, activation='relu')(f2)
# f2 = Dense(4, activation='sigmoid')(f2)
f2 = Dense(5, activation='sigmoid')(f2)

# model = Model(inputs = inputLayer, outputs=[f1, f2])
model = Model(inputs = inputLayer, outputs=f2)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 global_max_pooling2d (Globa  (None, 512)              0         
 lMaxPooling2D)                                                  
                                                                 
 dense (Dense)               (None, 2048)              1050624   
                                                                 
 dense_1 (Dense)             (None, 5)                 10245     
                                                                 
Total params: 15,775,557
Trainable params: 1,060,869
Non-trainable params: 14,714,688
_________________________________________

# Define Loss and Train

In [6]:

def localizationLoss(yTrue, yHat):
    deltaCoord = tf.reduce_sum(tf.square(yTrue[:,:2] - yHat[:,:2]))
    hTrue = yTrue[:,3] - yTrue[:,1]
    wTrue = yTrue[:,2] - yTrue[:,0]

    hPred = yHat[:,3] - yHat[:,1]
    wPred = yHat[:,2] - yHat[:,0]

    deltaSize = tf.reduce_sum(tf.square(wTrue - wPred) + tf.square(hTrue - hPred))

    return deltaCoord + deltaSize

classificationLoss = tf.keras.losses.BinaryCrossentropy()
def totalLoss(yTrue, yHat):
    print(yTrue, yHat)
    return classificationLoss(yTrue[0], yHat[0]) + localizationLoss(yTrue[1:], yHat[1:])
opt = tf.keras.optimizers.Adam()

In [7]:
class FaceDetection(Model):
    def __init__(self, model, **kwargs) -> None:
        super().__init__(**kwargs)
        self.model = model
    
    def compile(self, opt, classLoss, localLoss, **kwargs):
        super().compile(**kwargs)
        self.closs = classLoss
        self.lloss = localLoss
        self.opt = opt

    def train_step(self, batch, **kwargs):
        X, Y = batch

        with tf.GradientTape() as tape:
            classes, coords = self.model(X, training=True)
            print("out 1", Y[0][0], classes, coords)
            batch_closs = self.closs(Y[0], classes)
            batch_lloss = self.lloss(tf.cast(Y[1:], tf.float32), coords)
            print(batch_closs, batch_lloss)
            total_loss = 0.5 * batch_closs + batch_lloss
            print("total loss", total_loss)
            grad = tape.gradient(total_loss, self.model.trainable_variables)

        self.opt.apply_gradients(zip(grad, total_loss, self.model.trainable_variables))

        return {"total_loss": total_loss, "class_loss": batch_closs, "regress_loss": batch_lloss}
    
    def call(self, X, **kwargs):
        return self.model(X, **kwargs)

In [8]:
# model = FaceDetection(model)
# model.compile(opt, classificationLoss, localizationLoss)
# model.fit(train_dataset, epochs = 10, batch_size = 32)

In [9]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=totalLoss)
model.fit(train_dataset, epochs = 10, batch_size = 32)

Epoch 1/10
Tensor("Cast:0", shape=(None, 5), dtype=float32) Tensor("model/dense_1/Sigmoid:0", shape=(None, 5), dtype=float32)
Tensor("Cast:0", shape=(None, 5), dtype=float32) Tensor("model/dense_1/Sigmoid:0", shape=(None, 5), dtype=float32)
58/58 [==============================] - 221s 4s/step - loss: 43.4323
Epoch 2/10
58/58 [==============================] - 222s 4s/step - loss: 42.6455
Epoch 3/10
58/58 [==============================] - 221s 4s/step - loss: 62.5744
Epoch 4/10
58/58 [==============================] - 221s 4s/step - loss: 45.8374
Epoch 5/10
58/58 [==============================] - 220s 4s/step - loss: 45.9026
Epoch 6/10
58/58 [==============================] - 226s 4s/step - loss: 41.8201
Epoch 7/10
58/58 [==============================] - 226s 4s/step - loss: 37.6065
Epoch 8/10
58/58 [==============================] - 226s 4s/step - loss: 35.6224
Epoch 9/10
58/58 [==============================] - 229s 4s/step - loss: 35.5410
Epoch 10/10
58/58 [======================

In [10]:
def save_crop(imagePath, savedPath, left, top, right, bottom):
    img = Image.open(imagePath) 
    img_res = img.crop((left, top, right, bottom)) 
    img_res.save(savedPath) 

In [17]:
model.save('savemodel.h5')
test_image, test_image_data = preprocess_image("saved.jpeg", model_image_size = (IMAGE_SIZE, IMAGE_SIZE))
test_width, test_height = test_image.size
result = model.predict(test_image_data)
left, top, right, bottom = 1, int(result[0][2] * test_height), 100, int(result[0][4] * test_height)
print(left, top, right, bottom)
save_crop("saved.jpeg", "test_result.jpg", left, top, right, bottom)


1/1 [==============================] - 0s 117ms/step
1 68 50 162
